In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import statistics
from datetime import datetime
from pandas import Series
import warnings
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')

import seaborn as sns
import os
import statistics

from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from sklearn.metrics import median_absolute_error, mean_squared_error, mean_squared_log_error
from sklearn.model_selection import TimeSeriesSplit

from dateutil.relativedelta import relativedelta 
from scipy.optimize import minimize              

import statsmodels.formula.api as smf          
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs

from itertools import product      

from statsmodels.tsa.api import ExponentialSmoothing,SimpleExpSmoothing, Holt

from statsmodels.tsa.stattools import adfuller
from matplotlib.pylab import rcParams

from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA
from pmdarima.arima import auto_arima
import statsmodels.api as sm

In [2]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from pandas.plotting import register_matplotlib_converters
from sklearn.preprocessing import RobustScaler

%matplotlib inline
%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [4]:
def create_dataset(X, y, time_steps = 1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)        
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

In [5]:
from math import sqrt

def rmse(y_true, y_pred):
    squared_error = 0
    for i in range(len(y_true)):
        squared_error = squared_error + (y_true[i] - y_pred[i]) ** 2
    root_mean_squared_error = sqrt(squared_error / len(y_true))
    return root_mean_squared_error

In [6]:
def moving_avg(rounds):
    y_hat['avg'] = 0
    for i in range(len(Valid)):
        y_pred = 0
        for j in range(rounds):
            y_pred = y_pred + Train.footfall[len(Train) - (j + 1) * len(Valid) + i]
        y_hat['avg'][i] = y_pred / rounds

In [7]:
def test_stationary(timeseries):
    #Determine rolling statistics
    rolmean = pd.Series(timeseries).rolling(window = 28).mean()
    rolstd = pd.Series(timeseries).rolling(window = 28).std()
    
    #Plot rolling Statistics
    orig = plt.plot(timeseries, color = "blue", label = "Original")
    mean = plt.plot(rolmean, color = "red", label = "Rolling Mean")
    std = plt.plot(rolstd, color = "black", label = "Rolling Std")
    plt.legend(loc = "best")
    plt.title("Rolling Mean and Standard Deviation")
    plt.show(block = False)
    
    #Perform Dickey Fuller test
    print("Results of Dickey Fuller test: ")
    dftest = adfuller(timeseries, autolag = 'AIC')
    dfoutput = pd.Series(dftest[0:4], index = ['Test Statistics', 'p-value', '# Lag Used', 'Number of Observations Used'])
    
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)' %key] = value
    print(dfoutput)

In [8]:
def dm_test(actual_lst, pred1_lst, pred2_lst, h = 1, crit="MSE", power = 2):
    # Routine for checking errors
    def error_check():
        rt = 0
        msg = ""
        # Check if h is an integer
        if (not isinstance(h, int)):
            rt = -1
            msg = "The type of the number of steps ahead (h) is not an integer."
            return (rt,msg)
        # Check the range of h
        if (h < 1):
            rt = -1
            msg = "The number of steps ahead (h) is not large enough."
            return (rt,msg)
        len_act = len(actual_lst)
        len_p1  = len(pred1_lst)
        len_p2  = len(pred2_lst)
        # Check if lengths of actual values and predicted values are equal
        if (len_act != len_p1 or len_p1 != len_p2 or len_act != len_p2):
            rt = -1
            msg = "Lengths of actual_lst, pred1_lst and pred2_lst do not match."
            return (rt,msg)
        # Check range of h
        if (h >= len_act):
            rt = -1
            msg = "The number of steps ahead is too large."
            return (rt,msg)
        # Check if criterion supported
        if (crit != "MSE" and crit != "MAPE" and crit != "MAD" and crit != "poly"):
            rt = -1
            msg = "The criterion is not supported."
            return (rt,msg)  
        # Check if every value of the input lists are numerical values
        from re import compile as re_compile
        comp = re_compile("^\d+?\.\d+?$")  
        def compiled_regex(s):
            """ Returns True is string is a number. """
            if comp.match(s) is None:
                return s.isdigit()
            return True
        for actual, pred1, pred2 in zip(actual_lst, pred1_lst, pred2_lst):
            is_actual_ok = compiled_regex(str(abs(actual)))
            is_pred1_ok = compiled_regex(str(abs(pred1)))
            is_pred2_ok = compiled_regex(str(abs(pred2)))
            if (not (is_actual_ok and is_pred1_ok and is_pred2_ok)):  
                msg = "An element in the actual_lst, pred1_lst or pred2_lst is not numeric."
                rt = -1
                return (rt,msg)
        return (rt,msg)
    
    # Error check
    error_code = error_check()
    # Raise error if cannot pass error check
    if (error_code[0] == -1):
        raise SyntaxError(error_code[1])
        return
    # Import libraries
    from scipy.stats import t
    import collections
    import pandas as pd
    import numpy as np
    
    # Initialise lists
    e1_lst = []
    e2_lst = []
    d_lst  = []
    
    # convert every value of the lists into real values
    actual_lst = pd.Series(actual_lst).apply(lambda x: float(x)).tolist()
    pred1_lst = pd.Series(pred1_lst).apply(lambda x: float(x)).tolist()
    pred2_lst = pd.Series(pred2_lst).apply(lambda x: float(x)).tolist()
    
    # Length of lists (as real numbers)
    T = float(len(actual_lst))
    
    # construct d according to crit
    if (crit == "MSE"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append((actual - p1)**2)
            e2_lst.append((actual - p2)**2)
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "MAD"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(abs(actual - p1))
            e2_lst.append(abs(actual - p2))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "MAPE"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(abs((actual - p1)/actual))
            e2_lst.append(abs((actual - p2)/actual))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "poly"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(((actual - p1))**(power))
            e2_lst.append(((actual - p2))**(power))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)    
    
    # Mean of d        
    mean_d = pd.Series(d_lst).mean()
    
    # Find autocovariance and construct DM test statistics
    def autocovariance(Xi, N, k, Xs):
        autoCov = 0
        T = float(N)
        for i in np.arange(0, N-k):
              autoCov += ((Xi[i+k])-Xs)*(Xi[i]-Xs)
        return (1/(T))*autoCov
    gamma = []
    for lag in range(0,h):
        gamma.append(autocovariance(d_lst,len(d_lst),lag,mean_d)) # 0, 1, 2
    V_d = (gamma[0] + 2*sum(gamma[1:]))/T
    DM_stat=V_d**(-0.5)*mean_d
    harvey_adj=((T+1-2*h+h*(h-1)/T)/T)**(0.5)
    DM_stat = harvey_adj*DM_stat
    # Find p-value
    p_value = 2*t.cdf(-abs(DM_stat), df = T - 1)
    
    # Construct named tuple for return
    dm_return = collections.namedtuple('dm_return', 'DM p_value')
    
    rt = dm_return(DM = DM_stat, p_value = p_value)
    
    return rt

### Exclude Locations with Too Much Missing Data:

In [9]:
ff = pd.read_csv('stackfootfall.csv')
ff.timestamp = pd.to_datetime(ff.timestamp, format = '%Y-%m-%d %H:%M:%S') 

In [10]:
ff = ff.loc[(ff.timestamp >= '2018-01') & (ff.timestamp <= '2019-08-18 23:00')]

ff_count = ff.groupby('location')['footfall'].count() / max(ff.groupby('location')['footfall'].count())
limit1 = ff_count.loc[ff_count >= 0.6].to_frame()
ff_test = ff.loc[(ff.timestamp >= '2018-07-29') & (ff.timestamp <= '2019-08-18 23:00')]
ff_test_count = ff_test.groupby('location')['footfall'].count() / max(ff_test.groupby('location')['footfall'].count())
limit2 = ff_test_count.loc[ff_test_count >= 0.8].to_frame()
limit = pd.merge(limit1, limit2, how = 'inner', on = 'location')

ff = pd.merge(ff, limit, how = 'inner', on = 'location')
ff

,Unnamed: 0,timestamp,location,device,footfall,year,month,day,hour,day_of_week,footfall_x,footfall_y
0,1344,2018-01-01 00:00:00,5.0,1780.0,494.0,2018,1,1,0,1,0.892256,0.833766
1,1345,2018-01-01 01:00:00,5.0,1780.0,899.0,2018,1,1,1,1,0.892256,0.833766
2,1346,2018-01-01 02:00:00,5.0,1780.0,770.0,2018,1,1,2,1,0.892256,0.833766
3,1347,2018-01-01 03:00:00,5.0,1780.0,599.0,2018,1,1,3,1,0.892256,0.833766
4,1348,2018-01-01 04:00:00,5.0,1780.0,331.0,2018,1,1,4,1,0.892256,0.833766
...,...,...,...,...,...,...,...,...,...,...,...,...
5002195,233635,2019-08-18 19:00:00,1168.0,827.0,42.0,2019,8,18,19,7,0.619529,0.955844
5002196,233636,2019-08-18 20:00:00,1168.0,827.0,10.0,2019,8,18,20,7,0.619529,0.955844
5002197,233637,2019-08-18 21:00:00,1168.0,827.0,13.0,2019,8,18,21,7,0.619529,0.955844
5002198,233638,2019-08-18 22:00:00,1168.0,827.0,1.0,2019,8,18,22,7,0.619529,0.955844


In [11]:
pred_accuracy = pd.DataFrame(columns = ('location', 'Naive', 'Moving_Average', 'Expoential_Smoothing', 
                                        'SARIMA', 'LSTM'))

dmtest = pd.DataFrame(columns = ('location', 'nm', 'p_nm', 'ne', 'p_ne', 'ns', 'p_ns', 
                                 'nl', 'p_nl', 'me', 'p_me', 'ms', 'p_ms', 'ml', 'p_ml', 
                                 'es', 'p_es', 'el', 'p_el', 'sl', 'p_sl'))

In [17]:
ff.location.nunique()

array([   5.,    6.,    7.,   31.,   62.,   78.,   87.,   88.,   89.,
         91.,   92.,   97.,  108.,  111.,  113.,  114.,  115.,  116.,
        117.,  119.,  121.,  123.,  125.,  126.,  127.,  130.,  137.,
        138.,  145.,  146.,  152.,  155.,  164.,  165.,  178.,  185.,
        186.,  188.,  190.,  218.,  222.,  233.,  237.,  249.,  251.,
        255.,  266.,  268.,  271.,  272.,  273.,  281.,  300.,  305.,
        306.,  313.,  321.,  324.,  329.,  334.,  357.,  376.,  382.,
        383.,  385.,  388.,  389.,  390.,  393.,  409.,  411.,  414.,
        421.,  422.,  425.,  428.,  429.,  430.,  434.,  436.,  437.,
        439.,  440.,  441.,  443.,  445.,  448.,  450.,  452.,  463.,
        464.,  480.,  488.,  489.,  490.,  491.,  495.,  498.,  500.,
        501.,  502.,  503.,  509.,  510.,  511.,  512.,  514.,  515.,
        516.,  517.,  519.,  521.,  522.,  523.,  524.,  526.,  527.,
        528.,  530.,  531.,  536.,  537.,  541.,  542.,  544.,  546.,
        547.,  548.,

In [26]:
for loc in ff.location.unique()[150:170]:

    ff_loc = ff.loc[(ff.location <= loc) & (ff.location >= loc)][[
        'timestamp','footfall','year','month','day','hour','day_of_week']]    
    ff_loc.index = ff_loc.timestamp
    ff_loc = ff_loc.resample('H').mean()

    ff_loc['footfall'].replace(0, np.nan, inplace = True)
    ff_loc['footfall'] = ff_loc['footfall'].interpolate(method='linear')
    ff_loc = ff_loc.reset_index(level = ['timestamp'])
    ff_loc.timestamp = pd.to_datetime(ff_loc.timestamp, format = '%Y-%m-%d %H:%M:%S')

    temp = ff_loc[['timestamp', 'footfall']]
    temp.index = temp.timestamp
    temp['year'] = temp.timestamp.dt.year
    temp['month'] = temp.timestamp.dt.month
    temp['day'] = temp.timestamp.dt.day
    temp['hour'] = temp.timestamp.dt.hour
    temp['day_of_week'] = temp.timestamp.dt.dayofweek + 1

    Train = temp.loc[(temp.timestamp >= '2018-01-01 00:00') & (temp.timestamp <= '2019-07-28 23:00')]
    Valid = temp.loc[(temp.timestamp >= '2019-07-29 00:00') & (temp.timestamp <= '2019-08-18 23:00')]
    
    if len(Valid) > 0:
        train = Train.resample('D').mean()
        valid = Valid.resample('D').mean() 

        train['footfall'] = train['footfall'].interpolate(method='linear')
        valid['footfall'] = valid['footfall'].interpolate(method='linear')

        # Monday to Thursday Ratio
        mtt = Train.loc[(Train.day_of_week >= 1) & (Train.day_of_week <= 4)].groupby('hour')['footfall'].mean().to_frame()
        mtt['ratio'] = mtt['footfall']/mtt['footfall'].sum()
        mtt = mtt.reset_index(level = ['hour'])

        # Friday Ratio
        fri = Train.loc[(Train.day_of_week >= 5) & (Train.day_of_week <= 5)].groupby('hour')['footfall'].mean().to_frame()
        fri['ratio'] = fri['footfall']/fri['footfall'].sum()
        fri = fri.reset_index(level = ['hour'])

        # Sat Ratio
        sat = Train.loc[(Train.day_of_week >= 6) & (Train.day_of_week <= 6)].groupby('hour')['footfall'].mean().to_frame()
        sat['ratio'] = sat['footfall']/sat['footfall'].sum()
        sat = sat.reset_index(level = ['hour'])

        # Sun Ratio
        sun = Train.loc[(Train.day_of_week >= 7) & (Train.day_of_week <= 7)].groupby('hour')['footfall'].mean().to_frame()
        sun['ratio'] = sun['footfall']/sun['footfall'].sum()
        sun = sun.reset_index(level = ['hour'])

        ratio1 = mtt[['hour','ratio']]
        ratio1['day_of_week'] = 1
        ratio2 = mtt[['hour','ratio']]
        ratio2['day_of_week'] = 2
        ratio3 = mtt[['hour','ratio']]
        ratio3['day_of_week'] = 3
        ratio4 = mtt[['hour','ratio']]
        ratio4['day_of_week'] = 4
        ratio5 = fri[['hour','ratio']] 
        ratio5['day_of_week'] = 5
        ratio6 = sat[['hour','ratio']]
        ratio6['day_of_week'] = 6
        ratio7 = sun[['hour','ratio']]
        ratio7['day_of_week'] = 7

        ratio_week = ratio1.append(ratio2).append(ratio3).append(ratio4).append(ratio5).append(ratio6).append(ratio7)

        merge = pd.merge(Valid, ratio_week, on = ('hour','day_of_week'), how = 'left')

        # Naive Approach
        dd = np.asarray(Train['footfall'])
        y_hat = Valid.copy()
        y_hat['naive']= dd[len(dd)- 1]
        for i in range(len(Valid)): 
            y_hat['naive'][i]= dd[len(dd) - len(Valid) + i]
        naive_rmse = rmse(Valid.footfall, y_hat.naive)


        # Moving Average Approach    
        moving_avg(4)
        avg4_rmse = rmse(Valid.footfall, y_hat.avg)
        moving_avg(6)
        avg6_rmse = rmse(Valid.footfall, y_hat.avg)    
        moving_avg(8)
        avg8_rmse = rmse(Valid.footfall, y_hat.avg)
        moving_avg(10)
        avg10_rmse = rmse(Valid.footfall, y_hat.avg)

        avg_rmse = min(avg4_rmse, avg6_rmse, avg8_rmse, avg10_rmse) 
        ma_temp = [avg4_rmse, avg6_rmse, avg8_rmse, avg10_rmse] 
        windows = (ma_temp.index(avg_rmse) + 2) * 2   
        moving_avg(windows)


        # Exponential Smoothing
        y_hat2 = valid.copy()
        fit1 = ExponentialSmoothing(np.asarray(train.footfall), seasonal_periods = 7, trend = 'add', seasonal= 'add').fit()
        y_hat2['Holt_Winter'] = fit1.forecast(len(valid))

        pred_d = y_hat2.Holt_Winter.to_frame()
        pred_d = pred_d.reset_index(level = ['timestamp'])
        pred_d['month'] = pred_d.timestamp.dt.month
        pred_d['day'] = pred_d.timestamp.dt.day
        temp1 = pd.merge(merge, pred_d, on = ('month','day'), how = 'left')
        temp1['prediction'] = temp1['ratio'] * temp1['Holt_Winter'] * 24
        temp1.index = temp1.timestamp_x
        y_hat['Holt_Winter'] = temp1['prediction']
        holt_winter_rmse = rmse(Valid.footfall, y_hat['Holt_Winter'])

        # SARIMA
        fit1 = sm.tsa.statespace.SARIMAX(train.footfall, order = (1,1,3), seasonal_order =(1,1,3,7)).fit()
        y_hat2['SARIMA'] = fit1.predict(start = "2019-07-29 00:00", end = "2019-08-18 23:00", dynamic=True)

        pred_d = y_hat2['SARIMA'].to_frame()
        pred_d = pred_d.reset_index(level = ['timestamp'])
        pred_d['month'] = pred_d.timestamp.dt.month
        pred_d['day'] = pred_d.timestamp.dt.day
        temp2 = pd.merge(merge, pred_d, on = ('month','day'), how = 'left')
        temp2['prediction'] = temp2['ratio'] * temp2['SARIMA'] * 24
        temp2.index = temp2.timestamp_x
        y_hat['SARIMA'] = temp2['prediction']
        sarima_rmse = rmse(Valid.footfall, y_hat['SARIMA'])

        # LSTM 
        train = temp.loc[(temp.timestamp >= '2018-01-01 00:00') & (temp.timestamp <= '2019-07-28 23:00')][[
            'footfall','month','day','hour','day_of_week']]
        test = temp.loc[(temp.timestamp >= '2019-07-28 14:00') & (temp.timestamp <= '2019-08-18 23:00')][[
            'footfall','month','day','hour','day_of_week']]  

        ff_transformer = RobustScaler()
        ff_transformer = ff_transformer.fit(train[['footfall']])

        train['footfall'] = ff_transformer.transform(train[['footfall']])
        test['footfall'] = ff_transformer.transform(test[['footfall']])

        time_steps = 10
        X_train, y_train = create_dataset(train, train.footfall, time_steps)
        X_test, y_test = create_dataset(test, test.footfall, time_steps)

        model = keras.Sequential()
        model.add(
          keras.layers.Bidirectional(
            keras.layers.LSTM(
              units=128, 
              input_shape=(X_train.shape[1], X_train.shape[2])
            )
          )
        )
        model.add(keras.layers.Dropout(rate=0.2))
        model.add(keras.layers.Dense(units=1))
        model.compile(loss='mean_squared_error', optimizer='adam')    

        history = model.fit(
        X_train, y_train, 
        epochs = 30, 
        batch_size = 32, 
        validation_split = 0.1,
        shuffle = False
        )

        y_pred = model.predict(X_test)

        y_train_inv = ff_transformer.inverse_transform(y_train.reshape(1, -1))
        y_test_inv = ff_transformer.inverse_transform(y_test.reshape(1, -1))
        y_pred_inv = ff_transformer.inverse_transform(y_pred)

        y_hat['LSTM'] =  y_pred_inv.flatten()
        lstm_rmse = rmse(y_test_inv.flatten(), y_pred_inv.flatten())

        # save prediction
        pred_loc = y_hat[['naive','avg','Holt_Winter','SARIMA','LSTM']]
        pred_loc.to_csv( 'pred/'+ str(loc) + '.csv')

        # DM-Test
        dm_NM = dm_test(Valid.footfall, y_hat['naive'], y_hat['avg'], h = 1, crit="MSE")
        dm_NE = dm_test(Valid.footfall, y_hat['naive'], y_hat['Holt_Winter'], h = 1, crit="MSE")    
        dm_NS = dm_test(Valid.footfall, y_hat['naive'], y_hat['SARIMA'], h = 1, crit="MSE")  
        dm_NL = dm_test(Valid.footfall, y_hat['naive'], y_pred_inv.flatten(), h = 1, crit="MSE")   
        dm_ME = dm_test(Valid.footfall, y_hat['avg'], y_hat['Holt_Winter'], h = 1, crit="MSE")   
        dm_MS = dm_test(Valid.footfall, y_hat['avg'], y_hat['SARIMA'], h = 1, crit="MSE")   
        dm_ML = dm_test(Valid.footfall, y_hat['avg'],  y_pred_inv.flatten(), h = 1, crit="MSE")   
        dm_ES = dm_test(Valid.footfall, y_hat['Holt_Winter'], y_hat['SARIMA'], h = 1, crit="MSE")    
        dm_EL = dm_test(Valid.footfall, y_hat['Holt_Winter'], y_pred_inv.flatten(), h = 1, crit="MSE")  
        dm_SL = dm_test(Valid.footfall, y_hat['SARIMA'], y_pred_inv.flatten(), h = 1, crit="MSE")

        dmtest = dmtest.append({'location': loc, 'nm': dm_NM.DM, 'p_nm': dm_NM.p_value, 
                                'ne': dm_NE.DM, 'p_ne': dm_NE.p_value, 'ns': dm_NS.DM, 'p_ns': dm_NS.p_value, 
                                'nl': dm_NL.DM, 'p_nl': dm_NL.p_value, 'me': dm_ME.DM, 'p_me': dm_ME.p_value, 
                                'ms': dm_MS.DM, 'p_ms': dm_MS.p_value, 'ml': dm_ML.DM, 'p_ml': dm_ML.p_value, 
                                'es': dm_ES.DM, 'p_es': dm_ES.p_value, 'el': dm_EL.DM, 'p_el': dm_EL.p_value, 
                                'sl': dm_SL.DM, 'p_sl': dm_SL.p_value}, ignore_index=True)


        pred_accuracy = pred_accuracy.append({'location': loc, 'Naive': naive_rmse, 'Moving_Average': avg_rmse, 
                                            'Expoential_Smoothing': holt_winter_rmse, 'SARIMA': sarima_rmse, 
                                            'LSTM': lstm_rmse}, ignore_index=True)


E:\Anaconda3\envs\urbsim\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


Epoch 1/30
388/388 [==============================] - 10s 25ms/step - loss: 0.2320 - val_loss: 0.1322
Epoch 2/30
388/388 [==============================] - 8s 20ms/step - loss: 0.0694 - val_loss: 0.0955
Epoch 3/30
388/388 [==============================] - 8s 19ms/step - loss: 0.0482 - val_loss: 0.0763
Epoch 4/30
388/388 [==============================] - 7s 19ms/step - loss: 0.0369 - val_loss: 0.0629
Epoch 5/30
388/388 [==============================] - 8s 20ms/step - loss: 0.0330 - val_loss: 0.0548
Epoch 6/30
388/388 [==============================] - 8s 20ms/step - loss: 0.0299 - val_loss: 0.0556
Epoch 7/30
388/388 [==============================] - 8s 20ms/step - loss: 0.0282 - val_loss: 0.0536
Epoch 8/30
388/388 [==============================] - 8s 19ms/step - loss: 0.0272 - val_loss: 0.0609
Epoch 9/30
388/388 [==============================] - 8s 20ms/step - loss: 0.0255 - val_loss: 0.0580
Epoch 10/30
388/388 [==============================] - 8s 19ms/step - loss: 0.0255 - val_l

E:\Anaconda3\envs\urbsim\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


Epoch 1/30
384/384 [==============================] - 8s 21ms/step - loss: 0.2974 - val_loss: 0.1757
Epoch 2/30
384/384 [==============================] - 7s 19ms/step - loss: 0.1358 - val_loss: 0.1515
Epoch 3/30
384/384 [==============================] - 7s 19ms/step - loss: 0.0904 - val_loss: 0.1054
Epoch 4/30
384/384 [==============================] - 7s 19ms/step - loss: 0.0723 - val_loss: 0.1048
Epoch 5/30
384/384 [==============================] - 7s 19ms/step - loss: 0.0652 - val_loss: 0.1091
Epoch 6/30
384/384 [==============================] - 7s 19ms/step - loss: 0.0632 - val_loss: 0.0958
Epoch 7/30
384/384 [==============================] - 7s 19ms/step - loss: 0.0595 - val_loss: 0.1001
Epoch 8/30
384/384 [==============================] - 7s 19ms/step - loss: 0.0585 - val_loss: 0.0935
Epoch 9/30
384/384 [==============================] - 7s 19ms/step - loss: 0.0574 - val_loss: 0.1013
Epoch 10/30
384/384 [==============================] - 7s 19ms/step - loss: 0.0557 - val_lo

E:\Anaconda3\envs\urbsim\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


...

Epoch 1/30
388/388 [==============================] - 8s 21ms/step - loss: 0.2150 - val_loss: 0.1153
Epoch 2/30
388/388 [==============================] - 7s 19ms/step - loss: 0.0935 - val_loss: 0.0774
Epoch 3/30
388/388 [==============================] - 8s 19ms/step - loss: 0.0639 - val_loss: 0.0501
Epoch 4/30
388/388 [==============================] - 8s 19ms/step - loss: 0.0502 - val_loss: 0.0468
Epoch 5/30
388/388 [==============================] - 7s 19ms/step - loss: 0.0458 - val_loss: 0.0436
Epoch 6/30
388/388 [==============================] - 8s 19ms/step - loss: 0.0443 - val_loss: 0.0482
Epoch 7/30
388/388 [==============================] - 8s 20ms/step - loss: 0.0434 - val_loss: 0.0468
Epoch 8/30
388/388 [==============================] - 8s 19ms/step - loss: 0.0401 - val_loss: 0.0445
Epoch 9/30
388/388 [==============================] - 8s 19ms/step - loss: 0.0401 - val_loss: 0.0465
Epoch 10/30
388/388 [==============================] - 7s 19ms/step - loss: 0.0394 - val_lo

E:\Anaconda3\envs\urbsim\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


Epoch 1/30
388/388 [==============================] - 8s 21ms/step - loss: 0.1872 - val_loss: 0.0831
Epoch 2/30
388/388 [==============================] - 7s 19ms/step - loss: 0.0736 - val_loss: 0.0576
Epoch 3/30
388/388 [==============================] - 7s 19ms/step - loss: 0.0463 - val_loss: 0.0465
Epoch 4/30
388/388 [==============================] - 7s 19ms/step - loss: 0.0347 - val_loss: 0.0419
Epoch 5/30
388/388 [==============================] - 7s 19ms/step - loss: 0.0283 - val_loss: 0.0392
Epoch 6/30
388/388 [==============================] - 7s 19ms/step - loss: 0.0264 - val_loss: 0.0328
Epoch 7/30
388/388 [==============================] - 8s 20ms/step - loss: 0.0241 - val_loss: 0.0287
Epoch 8/30
388/388 [==============================] - 7s 19ms/step - loss: 0.0220 - val_loss: 0.0324
Epoch 9/30
388/388 [==============================] - 8s 19ms/step - loss: 0.0215 - val_loss: 0.0305
Epoch 10/30
388/388 [==============================] - 8s 20ms/step - loss: 0.0203 - val_lo

E:\Anaconda3\envs\urbsim\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


Epoch 1/30
384/384 [==============================] - 8s 21ms/step - loss: 0.1773 - val_loss: 0.0893
Epoch 2/30
384/384 [==============================] - 7s 19ms/step - loss: 0.0882 - val_loss: 0.0823
Epoch 3/30
384/384 [==============================] - 7s 19ms/step - loss: 0.0519 - val_loss: 0.0501
Epoch 4/30
384/384 [==============================] - 7s 19ms/step - loss: 0.0356 - val_loss: 0.0568
Epoch 5/30
384/384 [==============================] - 7s 19ms/step - loss: 0.0314 - val_loss: 0.0425
Epoch 6/30
384/384 [==============================] - 7s 19ms/step - loss: 0.0280 - val_loss: 0.0436
Epoch 7/30
384/384 [==============================] - 7s 19ms/step - loss: 0.0264 - val_loss: 0.0438
Epoch 8/30
384/384 [==============================] - 7s 19ms/step - loss: 0.0253 - val_loss: 0.0424
Epoch 9/30
384/384 [==============================] - 7s 19ms/step - loss: 0.0242 - val_loss: 0.0406
Epoch 10/30
384/384 [==============================] - 7s 19ms/step - loss: 0.0233 - val_lo

In [29]:
pred_accuracy = pred_accuracy.drop_duplicates(subset='location')
pred_accuracy

,location,Naive,Moving_Average,Expoential_Smoothing,SARIMA,LSTM
0,649.0,152.446447,126.736308,155.839159,123.081832,119.733967
1,657.0,115.470451,117.050864,149.063380,128.964391,90.084942
2,660.0,121.518827,98.962515,139.913709,136.558083,89.530249
3,705.0,271.650142,210.870070,251.154268,226.985080,184.235458
4,709.0,953.031099,718.172147,659.167248,1785.860576,442.999469
...,...,...,...,...,...,...
202,618.0,293.503782,293.328260,381.767447,266.996647,165.791184
203,620.0,84.110713,82.344358,73.814893,87.262827,87.529517
204,639.0,68.578061,53.407148,54.006851,55.602200,49.279187
205,642.0,43.010888,36.685520,55.225211,51.503534,31.890926


In [28]:
dmtest = dmtest.drop_duplicates(subset='location')
dmtest

,location,nm,p_nm,ne,p_ne,ns,p_ns,nl,p_nl,me,...,ms,p_ms,ml,p_ml,es,p_es,el,p_el,sl,p_sl
0,649.0,3.546045,4.276080e-04,-0.408963,6.827407e-01,4.093780,4.943738e-05,3.966777,8.343337e-05,-5.142984,...,1.593227,1.117374e-01,1.036569,3.004347e-01,6.100975,2.105414e-09,4.826646,1.844483e-06,0.518004,6.046832e-01
1,657.0,-0.275268,7.832231e-01,-4.652335,4.201824e-06,-2.602655,9.523463e-03,3.533407,4.480571e-04,-4.447895,...,-2.760637,5.979560e-03,3.911994,1.041156e-04,4.479601,9.264363e-06,5.988023,4.045203e-09,5.844916,9.120505e-09
2,660.0,2.136547,3.311676e-02,-1.871715,6.182624e-02,-1.547532,1.223640e-01,2.824611,4.921779e-03,-11.416242,...,-10.893467,5.907835e-25,2.534696,1.155654e-02,2.743397,6.297782e-03,12.156039,5.462983e-30,12.626319,6.112249e-32
3,705.0,4.126811,4.304850e-05,1.297692,1.949879e-01,2.859547,4.418702e-03,4.979523,8.774295e-07,-6.109131,...,-3.521727,4.677639e-04,3.671329,2.671043e-04,4.854340,1.614548e-06,6.625135,8.940581e-11,5.660855,2.533942e-08
4,709.0,2.343784,2.809796e-02,2.281848,3.207584e-02,-4.923383,5.640781e-05,2.991030,6.526270e-03,1.126822,...,-5.583782,1.109549e-05,2.430890,2.326723e-02,-5.604969,1.053732e-05,2.327757,2.908144e-02,6.325615,1.870059e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,618.0,0.018056,9.856015e-01,-6.101760,2.095810e-09,2.121741,3.434773e-02,7.533519,2.310444e-13,-9.723995,...,4.526586,7.490291e-06,9.465891,1.123787e-19,13.401975,3.075686e-35,12.386070,6.135109e-31,7.623441,1.240526e-13
203,620.0,0.420978,6.739513e-01,2.706819,7.024085e-03,-0.677611,4.983300e-01,-0.734189,4.631754e-01,3.162039,...,-2.018286,4.409179e-02,-1.681920,9.320510e-02,-5.671796,2.386399e-08,-3.723168,2.189722e-04,-0.070340,9.439512e-01
204,639.0,5.317610,1.584378e-07,4.435169,1.130763e-05,4.587744,5.663940e-06,5.413689,9.576277e-08,-0.407577,...,-1.785799,7.473416e-02,1.674110,9.473061e-02,-0.903625,3.666269e-01,2.049335,4.094750e-02,2.486040,1.324088e-02
205,642.0,2.341270,1.960777e-02,-6.545941,1.460417e-10,-4.639159,4.466977e-06,5.076409,5.424543e-07,-8.727257,...,-7.417905,5.097337e-13,2.353490,1.898198e-02,11.985763,2.717932e-29,16.389214,1.103553e-48,15.329677,8.432123e-44


In [30]:
dmtest.to_csv('dmtest2.csv')
pred_accuracy.to_csv('pred_accuracy2.csv')

In [31]:
dmtest1 = pd.read_csv('dmtest.csv')
dmtest1 = dmtest1.drop(columns=['Unnamed: 0'])

pred_accuracy1 = pd.read_csv('pred_accuracy.csv')
pred_accuracy1 = pred_accuracy1.drop(columns=['Unnamed: 0'])

In [180]:
dmtestz = dmtest.append(dmtest1)
dmtestz

,location,dm_nai_mov,p_nai_mov,dm_nai_es,p_nai_es,dm_nai_sa,p_nai_sa,dm_nai_ls,p_nai_ls,dm_mov_es,...,dm_mov_sa,p_mov_sa,dm_mov_ls,p_mov_ls,dm_es_sa,p_es_sa,dm_es_ls,p_es_ls,dm_sa_ls,p_sa_ls
0,123.0,5.572342,4.579726e-08,6.613142,1.185217e-10,6.646138,9.687066e-11,7.527820,3.351889e-13,4.922440,...,4.777178,2.485800e-06,8.146106,4.660093e-15,-1.420115,1.563399e-01,5.066094,6.170705e-07,5.054385,6.537654e-07
1,125.0,1.854522,6.676648e-02,0.824226,4.118758e-01,0.026970,9.785399e-01,1.712714,9.002724e-02,-2.625987,...,-2.579867,1.141633e-02,-0.097533,9.225083e-01,-1.793056,7.614635e-02,1.154491,2.511949e-01,1.532285,1.287755e-01
2,126.0,0.748011,4.548032e-01,4.046126,6.026210e-05,2.633514,8.710786e-03,4.564412,6.303586e-06,4.807093,...,3.306822,1.011074e-03,5.678262,2.303179e-08,-4.516985,7.823993e-06,1.363633,1.732929e-01,6.390069,3.783688e-10
3,127.0,2.001185,4.590922e-02,3.030513,2.567100e-03,3.258509,1.195720e-03,1.658482,9.784354e-02,1.311638,...,1.616946,1.065169e-01,-0.046836,9.626622e-01,2.270732,2.358519e-02,-1.263966,2.068276e-01,-1.643004,1.010071e-01
4,145.0,-0.941103,3.471038e-01,4.766637,2.455947e-06,5.428416,8.859192e-08,1.155144,2.485799e-01,4.761841,...,5.570164,4.150861e-08,3.470008,5.651810e-04,5.327175,1.507427e-07,-2.575802,1.028509e-02,-3.355312,8.526683e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,114.0,1.810261,7.085192e-02,-3.866473,1.249043e-04,-3.856509,1.299508e-04,5.834720,9.658441e-09,-8.627975,...,-8.815842,1.945979e-17,7.321437,9.793729e-13,0.432093,6.658587e-01,13.434180,2.233484e-35,13.308516,7.768267e-35
16,115.0,3.440270,6.294638e-04,6.238650,9.372059e-10,7.879858,2.042314e-14,10.560552,1.107468e-23,1.553167,...,3.261137,1.184921e-03,7.906980,1.683102e-14,6.995820,8.450157e-12,8.327324,7.884092e-16,7.163245,2.816957e-12
17,116.0,6.227919,9.987603e-10,7.551148,2.046147e-13,6.972595,9.825328e-12,7.887942,1.927978e-14,4.039571,...,2.635025,8.672652e-03,4.728078,2.947309e-06,-3.817114,1.518540e-04,2.382272,1.757683e-02,3.310813,9.970701e-04
18,117.0,11.512425,2.201661e-27,8.760171,2.989844e-17,9.758292,1.020031e-20,10.004916,1.298479e-21,-4.895982,...,-2.718983,6.774583e-03,4.612626,5.050583e-06,8.749134,3.254810e-17,7.236588,1.729888e-12,6.700677,5.574122e-11


In [181]:
pred_accuracyz = pred_accuracy.append(pred_accuracy1)
pred_accuracyz

,location,Naive,Moving_Average,Expoential_Smoothing,SARIMA,LSTM
0,123.0,351.532332,239.254357,174.336987,176.475121,103.383646
1,125.0,258.661651,185.829111,224.253530,257.283002,188.480157
2,126.0,84.002446,79.624075,48.370817,64.530111,43.037310
3,127.0,70.429155,61.147825,56.685093,55.404851,61.364009
4,145.0,91.320776,95.681339,74.905223,72.150681,85.798975
...,...,...,...,...,...,...
15,114.0,313.261116,292.347866,376.453230,375.977332,224.236904
16,115.0,398.359378,338.459993,311.973165,286.320937,217.402409
17,116.0,221.336806,152.766329,139.490316,144.624937,128.795894
18,117.0,165.508798,109.619888,125.616905,117.469585,84.322696


In [182]:
dmtestz.to_csv('dmtest.csv')
pred_accuracyz.to_csv('pred_accuracy.csv')